<a href="https://colab.research.google.com/github/raggi-eth/PNG_sequence_to_smooth_video/blob/main/safari_squad_habitats_smoothie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown ---
#@markdown <font color="white" size="+2">safari squad habitats smoothie</font></b><br>

In [ ]:
#@markdown ---
#@markdown <font color="white" size="+2">install dependencies</font></b><br>

!pip install ffmpeg-python
!git clone https://github.com/hzwer/arXiv2020-RIFE
!mkdir /content/arXiv2020-RIFE/train_log
%cd /content/arXiv2020-RIFE/train_log
!gdown --id 1APIzVeI-4ZZCEuIRE1m6WYfSCaOsi_7_
!7z e RIFE_trained_model_v3.6.zip
%cd /content/arXiv2020-RIFE/
!gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
!pip3 install -r requirements.txt

In [ ]:
import os
import builtins  
#import __builtin__ 

# Defines Helper Functions

def create_folder(directory: str):
  """
  Checks if a directory exists and if not creates it.
  :param directory: Required folder and path (directory)
  """
  is_folder = os.path.isdir(directory)
  if not is_folder:
    try:
        os.makedirs(directory)
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Error creating folder: ", e)
            raise

    print("Successfully created folder : ", directory)
  else:
    print("Folder exists. Script will use existing folder at : ", directory)

def create_temporary_folders():
  """
  Creates some temporary directories.
  """
  tmpDir = ("/content/tmp")
  inputDir = ("/content/output")

  # If folder doesn't exist, then create it.
  create_folder(tmpDir)
  create_folder(inputDir)

def get_api_key() -> str:
  """
  Checks if API-Key has been entered previously and if so
  returns it.
  :return: api key as string
  """
  file = pathlib.Path("/content/tmp/apikey.json")
  if file.exists () == False:
    sys.exit("Enter API-key in the top cell")

  my_key = ''
  with open('/content/tmp/apikey.json', 'r') as file_to_read:
    json_data = json.load(file_to_read)
    my_key = json_data["API_KEY"]
  return my_key

#@markdown ---
#@markdown <font color="white" size="+2">select folder</font></b><br>
upload_method = 'mount_google_drive' #@param ["browser_upload", "mount_google_drive"]
path_to_drive_folder = "safari/" #@param {type:"string"}

#@markdown ---
import os

create_temporary_folders()

if upload_method == "browser_upload":
    from google.colab import files
    %cd /content/input
    uploaded = files.upload()
    for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(
          name=fn, length=len(uploaded[fn])))
    input_folder="/content/input"

else:
    from google.colab import drive
    drive.mount('/content/drive')
    input_folder="/content/drive/MyDrive/{}".format(path_to_drive_folder)
    print ("/content/drive/MyDrive/{} is your input folder".format(path_to_drive_folder))
    #IDcacheFolder = "/content/drive/MyDrive/{}/tx-idCache/".format(path_to_drive_folder)
    
    #create_folder(IDcacheFolder)  

In [ ]:


#@markdown ---
#@markdown <font color="white" size="+2">press play</font></b><br>

import subprocess
import shutil
import os 
import ffmpeg
create_temporary_folders()
path = input_folder
inputfolders = os.listdir(path)
print("folders containing png sequences:") 
print(os.listdir(path))
print(type(inputfolders))
inputfolders.sort()
print("There are {} input folders".format(itt))

for i in inputfolders:
  print("copying folder #{}".format(i))
  src = "{}/{}".format(input_folder, i)
  dest = "/content/{}".format(i)
  destination = shutil.copytree(src, dest) 
  (
      ffmpeg
      .input('/content/{}/*.png'.format(i), pattern_type='glob', framerate=12)
      .output('/content/arXiv2020-RIFE/{}.mp4'.format(i))
      .run()
  )
  shutil.rmtree(dest, ignore_errors=True)
  print("Deleted {} directory successfully".format(i))
  %cd /content/arXiv2020-RIFE/
  !nvidia-smi
  !python /content/arXiv2020-RIFE/inference_video.py --video={i}.mp4 --exp=2 --skip --scale=0.5
  os.chdir('/content/')
  subprocess.call(["ffmpeg", "-y", "-i", '/content/arXiv2020-RIFE/{}_4X_48fps.mp4'.format(i), "-c:v", "libx264", "-preset", "slow", "-crf", "22", "-pix_fmt", "yuv420p", "-c:a", "libvo_aacenc", "-b:a", "128k", '/content/output/SafariHabitat#{}.mp4'.format(i)])  
  
  source = "/content/output/SafariHabitat#{}.mp4".format(i)
  destination = "{}/output".format(input_folder)
  create_folder(destination)
  try:
    shutil.copy(source, destination)
    print("File copied successfully.")
  except shutil.SameFileError:
      print("Source and destination represents the same file.")
  except PermissionError:
      print("Permission denied.")
  except:
      print("Error occurred while copying file.")

  

